In [1]:
import sys
sys.path.append('../')
from package_auto_assembler import (VersionHandler, \
    ImportMappingHandler, RequirementsHandler, MetadataHandler, \
        LocalDependaciesHandler, LongDocHandler, SetupDirHandler, \
            ReleaseNotesHandler, MkDocsHandler, PackageAutoAssembler)

### 1. Package versioning

Package versioning within paa is done based on semantic versioning.

`major.minor.patch`

By default, patch is updated, but the minor and major could also be update based on, for example, commit messages or manually from the log file. 

Package auto assembler does try to pull latest version from package storage, but in case of failure uses version logs.

#### Initialize VersionHandler

In [2]:
pv = VersionHandler(
    # required
    versions_filepath = '../tests/package_auto_assembler/other/lsts_package_versions.yml',
    log_filepath = '../tests/package_auto_assembler/other/version_logs.csv',
    # optional
    default_version = "0.0.1")

#### Add new package

In [3]:
pv.add_package(
    package_name = "new_package",
    # optional
    version = "0.0.1"
)

#### Update package version

In [4]:
pv.increment_patch(
    package_name = "new_package"
)
## for not tracked package
pv.increment_patch(
    package_name = "another_new_package",
    # optional
    default_version = "0.0.1"
)

There are no known versions of 'another_new_package', 0.0.1 will be used!


#### Display current versions and logs

In [5]:
pv.get_versions(
    # optional
    versions_filepath = '../tests/package_auto_assembler/other/lsts_package_versions.yml'
)

{'another_new_package': '0.0.1', 'new_package': '0.0.2'}

In [6]:
pv.get_version(
    package_name='new_package'
)

'0.0.2'

In [7]:
pv.get_logs(
    # optional
    log_filepath = '../tests/package_auto_assembler/other/version_logs.csv'
)

,Timestamp,Package,Version
0,2024-07-29 03:26:39,new_package,0.0.1
1,2024-07-29 03:26:40,new_package,0.0.2
2,2024-07-29 03:26:40,another_new_package,0.0.1


#### Flush versions and logs

In [8]:
pv.flush_versions()
pv.flush_logs()

#### Get latest available version with pip

In [9]:
pv.get_latest_pip_version(package_name = 'package-auto-assembler')

'0.3.1'

### 2. Import mapping

Install and import names of dependencies may vary. The mapping files maps import names to install names so that requirements extraction from `.py` files is possible.

#### Initialize ImportMappingHandler

In [10]:
im = ImportMappingHandler(
    # required
    mapping_filepath = "../env_spec/package_mapping.json"
)

#### Load package mappings

In [11]:
im.load_package_mappings(
    # optional
    mapping_filepath = "../env_spec/package_mapping.json"
)

{'PIL': 'Pillow',
 'bs4': 'beautifulsoup4',
 'fitz': 'PyMuPDF',
 'attr': 'attrs',
 'dotenv': 'python-dotenv',
 'googleapiclient': 'google-api-python-client',
 'google_auth_oauthlib': 'google-auth-oauthlib',
 'sentence_transformers': 'sentence-transformers',
 'flask': 'Flask',
 'stdlib_list': 'stdlib-list',
 'sklearn': 'scikit-learn',
 'yaml': 'pyyaml',
 'package_auto_assembler': 'package-auto-assembler',
 'git': 'gitpython'}

### 3. Extracting and merging requirements

Maintaining requirements is much simpler, when done automatically based on the `.py` files. 

The actual requirements files is still constructed. Standard libraries are not added, others are added with their versions, if specified. Local files are also used as dependencies, from which imports are extracted as well. 

For example:

```python
import os
import pandas
import attr #>=22.2.0
from .components.local_dep import *
```

Produces 

```
pandas
attrs >=22.2.0
yaml
```

as requirements file, where `yaml` is extracted from `local_dep.py` file.

Checking dependecies for vulnerabilities is usefull and it is done with `pip audit` which is integrated into the paa package and is used by default.

#### Initialize RequirementsHandler

In [12]:
rh = RequirementsHandler(
    # optional/required later
    module_filepath = "../tests/package_auto_assembler/other/example_module.py",
    package_mappings = {'PIL': 'Pillow',
                        'bs4': 'beautifulsoup4',
                        'fitz': 'PyMuPDF',
                        'attr': 'attrs',
                        'dotenv': 'python-dotenv',
                        'googleapiclient': 'google-api-python-client',
                        'sentence_transformers': 'sentence-transformers',
                        'flask': 'Flask',
                        'stdlib_list': 'stdlib-list',
                        'sklearn': 'scikit-learn',
                        'yaml': 'pyyaml'},
    requirements_output_path = "../tests/package_auto_assembler/other/",
    output_requirements_prefix = "requirements_",
    custom_modules_filepath = "../tests/package_auto_assembler/dependancies",
    python_version = '3.8',
    add_header = True
)

#### List custom modules for a given directory

In [13]:
rh.list_custom_modules(
    # optional
    custom_modules_filepath="../tests/package_auto_assembler/dependancies"
)

['example_local_dependacy_1', 'example_local_dependacy_2']

#### Check if module is a standard python library

In [14]:
rh.is_standard_library(
    # required
    module_name = 'example_local_dependacy_1',
    # optional
    python_version = '3.8'
)

False

In [15]:
rh.is_standard_library(
    # required
    module_name = 'logging',
    # optional
    python_version = '3.8'
)

True

#### Extract requirements from the module file

In [16]:
rh.extract_requirements(
    # optional
    module_filepath = "../tests/package_auto_assembler/other/example_module.py",
    custom_modules = ['example_local_dependacy_2', 'example_local_dependacy_1'],
    package_mappings = {'PIL': 'Pillow',
                        'bs4': 'beautifulsoup4',
                        'fitz': 'PyMuPDF',
                        'attr': 'attrs',
                        'dotenv': 'python-dotenv',
                        'googleapiclient': 'google-api-python-client',
                        'sentence_transformers': 'sentence-transformers',
                        'flask': 'Flask',
                        'stdlib_list': 'stdlib-list',
                        'sklearn': 'scikit-learn',
                        'yaml': 'pyyaml'},
    python_version = '3.8',
    add_header=True
)

['attrs>=22.2.0']

In [17]:
rh.requirements_list

['### example_module.py', 'attrs>=22.2.0']

#### Audit dependencies

In [18]:
rh.check_vulnerabilities(
    # optional if ran extract_requirements() before
    requirements_list = None,
    raise_error = True
)

No known vulnerabilities found



In [19]:
rh.vulnerabilities

[]

In [20]:
try:
    rh.check_vulnerabilities(
        # optional if ran extract_requirements() before
        requirements_list = ['attrs>=22.2.0', 'pandas', 'hnswlib==0.7.0'],
        raise_error = True
    )
except Exception as e:
    print(f"Error: {e}")

Found 1 known vulnerability in 1 package



Name    Version ID                  Fix Versions
------- ------- ------------------- ------------
hnswlib 0.7.0   GHSA-xwc8-rf6m-xr86

Error: Found vulnerabilities, resolve them or ignore check to move forwards!


In [21]:
rh.vulnerabilities

[{'name': 'hnswlib',
  'version': '0.7.0',
  'id': 'GHSA-xwc8-rf6m-xr86',
  'fix_versions': None}]

#### Save requirements to a file

In [22]:
rh.write_requirements_file(
    # optional/required later
    module_name = 'example_module',
    requirements = ['### example_module.py', 'attrs>=22.2.0'],
    output_path = "../tests/package_auto_assembler/other/",
    prefix = "requirements_"
)

#### Read requirements

In [23]:
rh.read_requirements_file(
    # required
    requirements_filepath = "../tests/package_auto_assembler/other/requirements_example_module.txt"
)

['attrs>=22.2.0']

### 4. Preparing metadata

Since all of the necessary information for building a package needs to be contained within main component `.py` file, basic metadata is provided with the use of `__package_metadata__` dictionary object, defined within that `.py` file. It is also used as a trigger for package building within paa pipeline.

#### Initializing MetadataHandler

In [24]:
mh = MetadataHandler(
    # optional/required later
    module_filepath = "../tests/package_auto_assembler/other/example_module.py"
)

#### Check if metadata is available

In [25]:
mh.is_metadata_available(
    # optional
    module_filepath = "../tests/package_auto_assembler/other/example_module.py"
)

True

#### Extract metadata from module

In [26]:
mh.get_package_metadata(
    # optional
    module_filepath = "../tests/package_auto_assembler/other/example_module.py"
)

{'author': 'Kyrylo Mordan',
 'author_email': 'parachute.repo@gmail.com',
 'version': '0.0.1',
 'description': 'A mock handler for simulating a vector database.',
 'keywords': ['python', 'vector database', 'similarity search']}

### 5. Merging local dependacies into single module

Package auto assembler creates `single module packages`, meaning that once package is built all of the object are imported from a single place. The packaging tool does allow for `local dependecies` which are `.py` files imported from specified dependencies directory and its subfolders. Packaging structure may look like the following:

```
packaging repo/
└src/
  ├ <package names>.py
  └ components
    ├local_dependecy.py
    └subdir_1
      └local_dependency_2.py 
```

During packaging process paa merges main module with its local dependies into a single file.

#### Initializing LocalDependaciesHandler

In [27]:
ldh = LocalDependaciesHandler(
    # required
    main_module_filepath = "../tests/package_auto_assembler/other/example_module.py",
    dependencies_dir = "../tests/package_auto_assembler/dependancies/",
    # optional
    save_filepath = "./combined_example_module.py"
)

#### Combine main module with dependacies

In [28]:
print(ldh.combine_modules(
    # optional
    main_module_filepath = "../tests/package_auto_assembler/other/example_module.py",
    dependencies_dir = "../tests/package_auto_assembler/dependancies/",
    add_empty_design_choices = False
)[0:1000])

"""
Mock Vector Db Handler

This class is a mock handler for simulating a vector database, designed primarily for testing and development scenarios.
It offers functionalities such as text embedding, hierarchical navigable small world (HNSW) search,
and basic data management within a simulated environment resembling a vector database.
"""

import logging
import json
import time
import attr #>=22.2.0
import sklearn

__design_choices__ = {}

@attr.s
class Shouter:

    """
    A class for managing and displaying formatted log messages.

    This class uses the logging module to create and manage a logger
    for displaying formatted messages. It provides a method to output
    various types of lines and headers, with customizable message and
    line lengths.
    """

    # Formatting settings
    dotline_length = attr.ib(default=50)

    # Logger settings
    logger = attr.ib(default=None)
    logger_name = attr.ib(default='Shouter')
    loggerLvl = attr.ib(default=logging.DEBUG)
    log

In [29]:
ldh.dependencies_names_list

['example_local_dependacy_2', 'example_local_dependacy_1', 'dep_from_bundle_1']

#### Save combined module

In [30]:
ldh.save_combined_modules(
    # optional
    combined_module = ldh.combine_modules(),
    save_filepath = "./combined_example_module.py"
)

### 6. Prepare README

Package description is based on `.ipynb` with same name as the `.py`. By default it is converted to markdown as is, but there is also an option to execute it.

In [31]:
import logging
ldh = LongDocHandler(
    # optional/required later
    notebook_path = "../tests/package_auto_assembler/other/example_module.ipynb",
    markdown_filepath = "../example_module.md",
    timeout = 600,
    kernel_name = 'python3',
    # logger
    loggerLvl = logging.DEBUG
)

#### Convert notebook to md without executing

In [32]:
ldh.convert_notebook_to_md(
    # optional
    notebook_path = "../tests/package_auto_assembler/other/example_module.ipynb",
    output_path = "../example_module.md"
)

Converted ../tests/package_auto_assembler/example_module.ipynb to ../example_module.md


#### Convert notebook to md with executing

In [33]:
ldh.convert_and_execute_notebook_to_md(
    # optional
    notebook_path = "../tests/package_auto_assembler/other/example_module.ipynb",
    output_path = "../example_module.md",
    timeout = 600,
    kernel_name = 'python3'
)

Converted and executed ../tests/package_auto_assembler/example_module.ipynb to ../example_module.md


#### Return long description

In [34]:
long_description = ldh.return_long_description(
    # optional
    markdown_filepath = "../example_module.md"
)

### 7. Assembling setup directory

Package are created following rather simple sequence of steps. At some point of the process a temporary directory is created to store the following files:

- `__init__.py` is a simple import from a single module
- `cli.py` is optional packaged cli tool
- `<package name>.py` is a single module with all of the local dependecies
- `README.md` is a package description file based on `.ipynb` file
- `setup.py` is a setup file for making a package

#### Initializing SetupDirHandler

In [35]:
sdh = SetupDirHandler(
    # required
    module_filepath = "../tests/package_auto_assembler/other/example_module.py",
    # optional/ required
    module_name = "example_module",
    metadata = {'author': 'Kyrylo Mordan',
                'version': '0.0.1',
                'description': 'Example module.',
                'long_description' : long_description,
                'keywords': ['python']},
    requirements = ['attrs>=22.2.0'],
    classifiers = ['Development Status :: 3 - Alpha',
                   'Intended Audience :: Developers',
                   'Intended Audience :: Science/Research',
                   'Programming Language :: Python :: 3',
                   'Programming Language :: Python :: 3.9',
                   'Programming Language :: Python :: 3.10',
                   'Programming Language :: Python :: 3.11',
                   'License :: OSI Approved :: MIT License',
                   'Topic :: Scientific/Engineering'],
    setup_directory = "./example_setup_dir"
)

#### Create empty setup dir

In [36]:
sdh.flush_n_make_setup_dir(
    # optional
    setup_directory = "./example_setup_dir"
)

#### Copy module to setup dir

In [37]:
sdh.copy_module_to_setup_dir(
    # optional
    module_filepath = "./combined_example_module.py",
    setup_directory = "./example_setup_dir"
)

#### Create init file

In [38]:
sdh.create_init_file(
    # optional
    module_name = "example_module",
    setup_directory = "./example_setup_dir"
)

#### Create setup file

In [39]:
sdh.write_setup_file(
    # optional
    module_name = "example_module",
    metadata = {'author': 'Kyrylo Mordan',
                'version': '0.0.1',
                'description': 'Example Module',
                'keywords': ['python']},
    requirements = ['attrs>=22.2.0'],
    classifiers = ['Development Status :: 3 - Alpha',
                   'Intended Audience :: Developers',
                   'Intended Audience :: Science/Research',
                   'Programming Language :: Python :: 3',
                   'Programming Language :: Python :: 3.9',
                   'Programming Language :: Python :: 3.10',
                   'Programming Language :: Python :: 3.11',
                   'License :: OSI Approved :: MIT License',
                   'Topic :: Scientific/Engineering'],
    setup_directory = "./example_setup_dir"
)

### 8. Creating release notes from commit messages

Package versioning could be enhanced with release notes. Since the tool is mainly meant for ci/cd, it takes advantage of commit messages to construct a release note for every version. 

Commit history is analysed from the last merge, if nothiong found then the next and the next, until at least one of `[<package name>]` labels are found within commit messages. They are bundled together to for a note, where each commit message or messages deliminated with `;` are turned in a list element. Previos notes are used to establish which part of commit history to use as a starting point.

Commit messages could also be used to increment version by something other then a default patch. 

- `[<package name>][..+]` increments patch (default behavior)
- `[<package name>][.+.]` increments minor
- `[<package name>][+..]` increments major
- `[<package name>][0.1.2]` forces specific version `0.1.2`

\* First release within new packaging repo may struggle to extract release note since commit messages are only analysed from merges in the commit history. 


In [2]:
rnh = ReleaseNotesHandler(
    # path to existing or new release notes file
    filepath = '../tests/package_auto_assembler/other/release_notes.md',
    # name of label in commit message [example_module] for filter
    label_name = 'example_module',
    # new version to be used in release notes
    version = '0.0.1'
)

No relevant commit messages found!
..trying depth 2 !
No relevant commit messages found!
No messages to clean were provided


##### - overwritting commit messages from example

In [15]:
# commit messages from last merge
rnh.commit_messages

['fixing paa tests',
 'fixing paa tests',
 'fixing paa tests',
 '[package_auto_assembler] increasing default max search depth for commit history to 5',
 'fixing mocker-db release notes',
 'Update package version tracking files',
 'Update README',
 'Update requirements']

In [16]:
example_commit_messages = [
    '[example_module] usage example for initial release notes; bugfixes for RNH',
    '[BUGFIX] missing parameterframe usage example and reduntant png file',
    '[example_module][0.1.2] initial release notes handler',
    'Update README',
    'Update requirements'
]
rnh.commit_messages = example_commit_messages

##### - internal methods that run on intialiazation of ReleaseNotesHandler

In [17]:
# get messages relevant only for label
rnh._filter_commit_messages_by_package()
print("Example filtered_messaged:")
print(rnh.filtered_messages)

# clean messages
rnh._clean_and_split_commit_messages()
print("Example processed_messages:")
print(rnh.processed_messages)

Example filtered_messaged:
['[example_module] usage example for initial release notes; bugfixes for RNH', '[example_module][0.1.2] initial release notes handler']
Example processed_messages:
['usage example for initial release notes', 'bugfixes for RNH', 'initial release notes handler']


##### - get version update from relevant messages

In [18]:
version_update = rnh.extract_latest_version()
print(f"Example version_update: {version_update}")

Example version_update: 0.1.2


##### - get latest version from relevant release notes

In [5]:
latest_version = rnh.extract_latest_version()
print(f"Example latest_version: {latest_version}")

Example latest_version: 0.1.2


##### - augment existing release note with new entries or create new

In [20]:
# augment existing release note with new entries or create new
rnh.create_release_note_entry(
    # optional
    existing_contents=rnh.existing_contents,
    version=rnh.version,
    new_messages=rnh.processed_messages
)
print("Example processed_note_entries:")
print(rnh.processed_note_entries)

Example processed_note_entries:
['# Release notes\n', '\n', '### 0.1.2\n', '\n', '    - usage example for initial release notes\n', '\n', '    - bugfixes for RNH\n', '\n', '    - initial release notes handler\n', '\n', '### 0.0.1\n', '\n', '    - initial version of example_module\n']


##### - saving updated relese notes

In [21]:
rnh.existing_contents

['# Release notes\n',
 '\n',
 '### 0.1.2\n',
 '\n',
 '    - usage example for initial release notes\n',
 '    - bugfixes for RNH\n',
 '    - initial release notes handler\n',
 '### 0.1.2\n',
 '\n',
 '    - usage example for initial release notes\n',
 '\n',
 '    - bugfixes for RNH\n',
 '\n',
 '    - initial release notes handler\n',
 '\n',
 '### 0.0.1\n',
 '\n',
 '    - initial version of example_module\n']

In [22]:
rnh.save_release_notes()

In [23]:
# updated content
rnh.get_release_notes_content()

['# Release notes\n',
 '\n',
 '### 0.1.2\n',
 '\n',
 '    - usage example for initial release notes\n',
 '\n',
 '    - bugfixes for RNH\n',
 '\n',
 '    - initial release notes handler\n',
 '\n',
 '### 0.0.1\n',
 '\n',
 '    - initial version of example_module\n']

### 9. Making a package

Main wrapper for the package integrates described above components into a class that could be used to build package building pipelines within python scripts. 

To simplify usage [cli interface](https://kiril-mordan.github.io/reusables/package_auto_assembler/cli/) is recomended instead. 

#### Initializing PackageAutoAssembler

In [49]:
paa = PackageAutoAssembler(
    # required
    module_name = "example_module",
    module_filepath  = "../tests/package_auto_assembler/other/example_module.py",
    # optional
    mapping_filepath = "../env_spec/package_mapping.json",
    dependencies_dir = "../tests/package_auto_assembler/dependancies/",
    example_notebook_path = "./mock_vector_database.ipynb",
    versions_filepath = '../tests/package_auto_assembler/other/lsts_package_versions.yml',
    log_filepath = '../tests/package_auto_assembler/other/version_logs.csv',
    setup_directory = "./example_module",
    release_notes_filepath = "../tests/package_auto_assembler/other/release_notes.md",
    classifiers = ['Development Status :: 3 - Alpha',
                    'Intended Audience :: Developers',
                    'Intended Audience :: Science/Research',
                    'Programming Language :: Python :: 3',
                    'Programming Language :: Python :: 3.9',
                    'Programming Language :: Python :: 3.10',
                    'Programming Language :: Python :: 3.11',
                    'License :: OSI Approved :: MIT License',
                    'Topic :: Scientific/Engineering'],
    requirements_list = [],
    execute_readme_notebook = True,
    python_version = "3.8",
    version_increment_type = "patch",
    default_version = "0.0.1",
    check_vulnerabilities = True,
    add_requirements_header = True
)

#### Add metadata from module

In [50]:
paa.add_metadata_from_module(
    # optional
    module_filepath  = "../tests/package_auto_assembler/example_module.py"
)

Adding metadata ...


#### Add or update version

In [51]:
paa.add_or_update_version(
    # overwrites auto mode (not suggested)
    version_increment_type = "patch",
    version = "1.2.6",
    # optional
    module_name = "example_module",
    versions_filepath = '../tests/package_auto_assembler/lsts_package_versions.yml',
    log_filepath = '../tests/package_auto_assembler/version_logs.csv'
)

Incrementing version ...
No relevant commit messages found!
..trying depth 2 !
No relevant commit messages found!
No messages to clean were provided
There are no known versions of 'example_module', 1.2.6 will be used!


#### Add release notes from commit messages

In [52]:
paa.add_or_update_release_notes(
    # optional
    filepath="../tests/package_auto_assembler/release_notes.md",
    version=paa.metadata['version']
)

Updating release notes ...


#### Prepare setup directory

In [53]:
paa.prep_setup_dir()

Preparing setup directory ...


#### Merge local dependacies

In [54]:
paa.merge_local_dependacies(
    # optional
    main_module_filepath = "../tests/package_auto_assembler/example_module.py",
    dependencies_dir= "../tests/package_auto_assembler/dependancies/",
    save_filepath = "./example_module/example_module.py"
)

Merging ../tests/package_auto_assembler/example_module.py with dependecies from ../tests/package_auto_assembler/dependancies/ into ./example_module/example_module.py


#### Add requirements from module

In [55]:
paa.add_requirements_from_module(
    # optional
    module_filepath = "../tests/package_auto_assembler/example_module.py",
    import_mappings = {'PIL': 'Pillow',
                        'bs4': 'beautifulsoup4',
                        'fitz': 'PyMuPDF',
                        'attr': 'attrs',
                        'dotenv': 'python-dotenv',
                        'googleapiclient': 'google-api-python-client',
                        'sentence_transformers': 'sentence-transformers',
                        'flask': 'Flask',
                        'stdlib_list': 'stdlib-list',
                        'sklearn': 'scikit-learn',
                        'yaml': 'pyyaml',
                        'git' : 'gitpython'}
)

Adding requirements from ../tests/package_auto_assembler/example_module.py
No known vulnerabilities found



In [56]:
paa.requirements_list

['### example_module.py', 'attrs>=22.2.0']

#### Make README out of example notebook

In [57]:
paa.add_readme(
    # optional
    example_notebook_path = "../tests/package_auto_assembler/example_module.ipynb",
    output_path = "./example_module/README.md",
    execute_notebook=False,
)

Adding README from ../tests/package_auto_assembler/example_module.ipynb to ./example_module/README.md


#### Prepare setup file

In [58]:
paa.prep_setup_file(
    # optional
    metadata = {'author': 'Kyrylo Mordan',
                'version': '0.0.1',
                'description': 'Example module',
                'keywords': ['python']},
    requirements = ['### example_module.py',
                    'attrs>=22.2.0'],
    classifiers = ['Development Status :: 3 - Alpha',
                    'Intended Audience :: Developers',
                    'Intended Audience :: Science/Research',
                    'Programming Language :: Python :: 3',
                    'Programming Language :: Python :: 3.9',
                    'Programming Language :: Python :: 3.10',
                    'Programming Language :: Python :: 3.11',
                    'License :: OSI Approved :: MIT License',
                    'Topic :: Scientific/Engineering'],
    cli_module_filepath = "../tests/package_auto_assembler/cli.py"

)

Preparing setup file for None package ...


#### Make package

In [59]:
paa.make_package(
    # optional
    setup_directory = "./example_module"
)

Making package from ./example_module ...


CompletedProcess(args=['python', './example_module/setup.py', 'sdist', 'bdist_wheel'], returncode=0, stdout="running sdist\nrunning egg_info\nwriting example_module.egg-info/PKG-INFO\nwriting dependency_links to example_module.egg-info/dependency_links.txt\nwriting entry points to example_module.egg-info/entry_points.txt\nwriting requirements to example_module.egg-info/requires.txt\nwriting top-level names to example_module.egg-info/top_level.txt\nreading manifest file 'example_module.egg-info/SOURCES.txt'\nwriting manifest file 'example_module.egg-info/SOURCES.txt'\nrunning check\ncreating example_module-0.0.1\ncreating example_module-0.0.1/example_module\ncreating example_module-0.0.1/example_module.egg-info\ncopying files to example_module-0.0.1...\ncopying example_module/__init__.py -> example_module-0.0.1/example_module\ncopying example_module/cli.py -> example_module-0.0.1/example_module\ncopying example_module/example_module.py -> example_module-0.0.1/example_module\ncopying exa

### 10. Making simple MkDocs site

Package documentation can be presented in a form of mkdocs static site, which could be either served or deployed to something like github packages. 

Main module docstring is used as intro package that contains something like optional pypi and license badges. Package description and realease notes are turned into separate tabs. Png with diagrams for example could be provided and displayed as their own separate tabs as well.
 
The one for this package can be seen [here](https://kiril-mordan.github.io/reusables/package_auto_assembler/)

##### - preparing inputs

In [60]:
package_name = "example_module"

module_content = LongDocHandler().read_module_content(filepath=f"../tests/package_auto_assembler/{package_name}.py")

docstring = LongDocHandler().extract_module_docstring(module_content=module_content)
pypi_link = LongDocHandler().get_pypi_badge(module_name=package_name)


docs_file_paths = {
    "../example_module.md" : "usage-examples.md",
    '../tests/package_auto_assembler/release_notes.md' : 'release_notes.md'
}

In [61]:
mdh = MkDocsHandler(
    # required
    ## name of the package to be displayed
    package_name = package_name,
    ## dictionary of markdown files, with path as keys
    docs_file_paths = docs_file_paths,
    # optional
    ## module docstring to be displayed in the index
    module_docstring = docstring,
    ## pypi badge to be displayed in the index
    pypi_badge = pypi_link,
    ## license badge to be displayed in the index
    license_badge="[![License](https://img.shields.io/github/license/Kiril-Mordan/reusables)](https://github.com/Kiril-Mordan/reusables/blob/main/LICENSE)",
    ## name of the project directory
    project_name = "temp_project")

##### - preparing site

In [62]:
mdh.create_mkdocs_dir()
mdh.move_files_to_docs()
mdh.generate_markdown_for_images()
mdh.create_index()
mdh.create_mkdocs_yml()
mdh.build_mkdocs_site()

Created new MkDocs dir: temp_project
Copied ../example_module.md to temp_project/docs/usage-examples.md
Copied ../tests/package_auto_assembler/release_notes.md to temp_project/docs/release_notes.md
index.md has been created with site_name: example-module
mkdocs.yml has been created with site_name: Example module
Custom CSS created at temp_project/docs/css/extra.css


INFO    -  Cleaning site directory
INFO    -  Building documentation to directory: /home/kyriosskia/Documents/nlp/reusables/example_notebooks/temp_project/site
INFO    -  Documentation built in 0.12 seconds


##### - test runing site

In [63]:
mdh.serve_mkdocs_site()